In [3]:
import pandas as pd

In [4]:
df_merge = pd.read_parquet('../data/train/merged_data.parquet')

In [5]:
df_train = pd.read_parquet('../data/train/train.parquet')

In [6]:
import json, re

In [7]:
def get_transformed_attribute(df):
    
    for row in df[:]['characteristic_attributes_mapping']:
        #print(row)
        result_str = ""
        row_dct = json.loads(row)


        items = list(row_dct.items())
        items.sort()                
        for key, val in items:
            result_str += key + " "
            val_str = " ".join(val)
            result_str += val_str
            result_str += " "
        result_str = re.sub(r'[^A-zА-я\s\d][\\\^]?', '', result_str)
        result_str = re.sub(r'\s{2,}', ' ', result_str)
    return result_str

In [8]:
def get_row_by_id(df_merge, id):
    return df_merge[df_merge['variantid'] == id]

In [9]:
for id1, id2, target in zip(df_train['variantid1'][:200], df_train['variantid2'][:200], df_train['target'][:200]):
    row1 = get_row_by_id(df_merge, id1)
    row2 = get_row_by_id(df_merge, id2)
    print("ID:", id1, id2)
    print("Таргет:", target)
    print("Атрибуты:")
    attribute1 = get_transformed_attribute(row1)
    attribute2 = get_transformed_attribute(row2)
    print("товар 1:", attribute1)
    print("товар 2:", attribute2)
    print("=================================")

ID: 1447875869 1447872068
Таргет: 1
Атрибуты:
товар 1: Бренд в одежде и обуви Uniqlo Декоративные элементы Разрезы Длина рукава Длинный Коллекция Осеньзима 2023 Материал Полиэфирное волокно Модель вязаная Параметры модели на фото 866088 Пол Женский Размер на модели 42 Размер производителя 3XL Рисунок Камуфляж Российский размер 52 Рост 150180 Рост модели на фото 168 см Сезон Зима Серия в одежде и обуви Harpers Bazaar журнал Состав материала 6535 Спортивное назначение Туризм Стиль Вечернее Странаизготовитель Япония Тип Свитер Тип застежки Без застежки Тип упаковки одежды Чехол Цвет товара светлобежевый 
товар 2: Бренд в одежде и обуви Uniqlo Декоративные элементы Разрезы Длина рукава Длинный Коллекция Осеньзима 2023 Материал Полиэфирное волокно Модель вязаная Параметры модели на фото 866088 Пол Женский Размер на модели 42 Размер производителя 3XL Рисунок Камуфляж Российский размер 52 Рост 150180 Рост модели на фото 168 см Сезон Зима Серия в одежде и обуви Harpers Bazaar журнал Состав мат

схожесть двух строк

In [10]:
def get_attr_simularity(attribute1, attribute2):
    pass

In [11]:
def get_uniq_attributes(df):
    """
    Функция возвращает dict из уникальных атрибутов

    return: dict

    """
    result_dict = dict()
    for row in df[:]['characteristic_attributes_mapping']:
        #print(row)
        row_dct = json.loads(row)


        for key, val in row_dct.items():
            if result_dict.get(key) is None:
                result_dict[key] = 1

    return result_dict


In [12]:
uniq_attr_count = get_uniq_attributes(df_merge)

In [39]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
# Загрузка другой модели эмбеддингов
model = SentenceTransformer('sergeyzh/rubert-tiny-turbo')

# Список уникальных атрибутов
attribute_names = [
    "способ применения", "способ назначения", 
    "количество в упаковке", "количество в упаковке, штук",
    "Длина по горизонтали, см", "Длина по вертикали, см",
    "Фирма", "гарантия",
    # добавьте остальные атрибуты...
]

attribute_names = list(uniq_attr_count.keys())[17:20]
print(attribute_names)
print(len(attribute_names))

# Преобразуем названия атрибутов в эмбеддинги
def preprocess_text(text):
    # Удаляем цифры и пунктуацию, приводим к нижнему регистру
    text = re.sub(r'\d+', '', text)  # Удаляем цифры
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем пунктуацию
    text = text.strip().lower()  # Приводим к нижнему регистру и убираем пробелы
    #return text
    # Лемматизация и приведение к единственному числу
    words = text.split()
    normalized_words = []
    for word in words:
        #print(word)
        p = morph.parse(word)[0]
        #print(p)
        # Приведение к лемме и единственному числу (если возможно)
        normalized_word = p.normal_form
        #print(normalized_word)
        # if 'NOUN' in p.tag:  # Если слово - существительное
        #     if p.tag.number == 'plur':  # Если слово во множественном числе
        #         singular = p.inflect({'sing'})
        #         if singular:
        #             normalized_word = singular.word
        normalized_words.append(normalized_word)
    
    # Сортировка слов в алфавитном порядке
    #normalized_words = sorted(normalized_words)
    
    # Объединяем обратно в строку
    processed_text = ' '.join(normalized_words)
    
    return processed_text

processed_attribute_names = [preprocess_text(attr) for attr in attribute_names]
print(processed_attribute_names)
# Преобразуем названия атрибутов в эмбеддинги
embeddings = model.encode(processed_attribute_names)

# Нормализуем эмбеддинги
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Применение кластеризации DBSCAN с косинусным расстоянием
clustering = DBSCAN(eps=0.1, min_samples=1, metric='cosine').fit(embeddings)

clusters = clustering.labels_

# Создаем DataFrame для наглядности
df = pd.DataFrame({
    'attribute': processed_attribute_names,
    'cluster': clusters
})

# Группируем атрибуты по кластерам
grouped_attributes = df.groupby('cluster')['attribute'].apply(list)

# Выводим результаты
for cluster_id, attributes in grouped_attributes.items():
    if cluster_id != -1:  # -1 означает шум (атрибуты, которые не были сгруппированы)
        print(f"Cluster {cluster_id}: {attributes}")


['Размеры, мм', 'Ширина, см', 'Состав ниток']
3
размеры
Parse(word='размеры', tag=OpencorporaTag('NOUN,inan,masc plur,nomn'), normal_form='размер', score=0.657894, methods_stack=((DictionaryAnalyzer(), 'размеры', 34, 6),))
размер
мм
Parse(word='мм', tag=OpencorporaTag('INTJ'), normal_form='мм', score=0.07692307692307693, methods_stack=((DictionaryAnalyzer(), 'мм', 21, 0),))
мм
ширина
Parse(word='ширина', tag=OpencorporaTag('NOUN,inan,femn,Sgtm sing,nomn'), normal_form='ширина', score=0.956521, methods_stack=((DictionaryAnalyzer(), 'ширина', 30, 0),))
ширина
см
Parse(word='см', tag=OpencorporaTag('VERB,impf,tran sing,impr,excl,Abbr'), normal_form='смотреть', score=0.928571, methods_stack=((DictionaryAnalyzer(), 'см', 2907, 14),))
смотреть
состав
Parse(word='состав', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='состав', score=0.734693, methods_stack=((DictionaryAnalyzer(), 'состав', 34, 3),))
состав
ниток
Parse(word='ниток', tag=OpencorporaTag('NOUN,inan,femn plur,gent'),

In [29]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('sergeyzh/rubert-tiny-turbo')

sentences = ['товар цвет', ]
embeddings = model.encode(sentences)
#print(embeddings)
print(torch.cosine_similarity(torch.from_numpy(embeddings[2]).unsqueeze(0), torch.from_numpy(embeddings[3]).unsqueeze(0)))
print(util.dot_score(embeddings, embeddings))

tensor([0.8645])
tensor([[1.0000, 0.7489, 0.8514, 0.8229, 0.7410, 0.7480, 0.7765, 0.6858, 0.8506,
         0.8600, 0.6850, 0.8013, 0.8050],
        [0.7489, 1.0000, 0.7518, 0.7782, 0.7324, 0.7576, 0.7180, 0.8403, 0.7813,
         0.7445, 0.8356, 0.6902, 0.7118],
        [0.8514, 0.7518, 1.0000, 0.8645, 0.7794, 0.7976, 0.8012, 0.7080, 0.8645,
         0.8356, 0.7127, 0.8166, 0.8209],
        [0.8229, 0.7782, 0.8645, 1.0000, 0.7961, 0.8169, 0.8291, 0.7194, 0.8870,
         0.8078, 0.7204, 0.8039, 0.8129],
        [0.7410, 0.7324, 0.7794, 0.7961, 1.0000, 0.8093, 0.7549, 0.6795, 0.8167,
         0.7557, 0.6870, 0.7588, 0.7695],
        [0.7480, 0.7576, 0.7976, 0.8169, 0.8093, 1.0000, 0.7963, 0.7196, 0.8149,
         0.7428, 0.7521, 0.7184, 0.7338],
        [0.7765, 0.7180, 0.8012, 0.8291, 0.7549, 0.7963, 1.0000, 0.6814, 0.8342,
         0.7602, 0.6938, 0.7629, 0.7706],
        [0.6858, 0.8403, 0.7080, 0.7194, 0.6795, 0.7196, 0.6814, 1.0000, 0.7143,
         0.6994, 0.8926, 0.6451, 0.6645],